In [1]:
# # We use uv for faster installation
# !pip install uv
# !uv pip install -q autogluon --system
# !uv pip install -q autogluon.timeseries --system
# !uv pip uninstall -q torchaudio torchvision torchtext --system # fix incompatible package versions on Colab

In [2]:
import numpy as np
import pandas as pd
import polars as pl
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [3]:
file_path = "https://github.com/davibarreira/mosqlimate-sprint/raw/main/data/dataset_complete_dengue_uf.parquet"
data = pl.read_parquet(file_path)

In [4]:
data = data.rename({'casos': 'target'})

In [5]:
exo = ['precip_min','precip_max','temp_min','temp_max','pressure_min','pressure_max','population']
train_1 = data.filter(pl.col('train_1'))[['uf','date','target',*exo]]
train_1 = TimeSeriesDataFrame.from_data_frame(train_1.to_pandas(),id_column='uf',timestamp_column='date')
test_1 =  data.filter(
    pl.col('target_1') | (
        # ~pl.col('train_1') &
        (pl.col('date') < np.max(np.array(data.filter(pl.col('target_1')).sort('date')['date'])))
    )
)[['uf','date','target',*exo]]
test_1 = TimeSeriesDataFrame.from_data_frame(test_1.to_pandas(),id_column='uf',timestamp_column='date')

end_1 = np.array(data.filter(pl.col('train_1')).sort('date')['epiweek'])[-1]
end_2 = np.array(data.filter(pl.col('target_1')).sort('date')['epiweek'])[-1]

prediction_length = 68

In [6]:
predictor = TimeSeriesPredictor(prediction_length=prediction_length).fit(
    train_data=train_1,
    hyperparameters={
        "Chronos": [
            {"model_path": "bolt_small", "ag_args": {"name_suffix": "ZeroShot"}},
            {"model_path": "bolt_small", "fine_tune": True, "ag_args": {"name_suffix": "FineTuned"}},
            # {"model_path": "bolt_small",
            #  "covariate_regressor": "CAT",
            #  "target_scaler": "standard",
            #  "fine_tune": True,
            #  "ag_args": {"name_suffix": "FineTuned-Covariates"}},
        ]
    },
    time_limit=60,  # time limit in seconds
    enable_ensemble=False,
    presets='fast_training'
)

Beginning AutoGluon training... Time limit = 60s
AutoGluon will save models to '/Users/davibarreira/Documents/DSProjects/Mosqlimate/jbd-mosqlimate-sprint/notebooks/AutogluonModels/ag-20250730_143149'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.11
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:33 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T8122
CPU Count:          8
GPU Count:          1
Memory Avail:       4.41 GB / 16.00 GB (27.6%)
Disk Space Avail:   382.92 GB / 926.35 GB (41.3%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'ZeroShot'},
                                  'model_path': 'bolt_small'},
                                 {'ag_args': {'name_suffix': 'FineTuned'},
                                  'fine_tune': True,
     

In [7]:
predictor.leaderboard(test_1)


Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,ChronosFineTuned[bolt_small],-0.534544,-0.647891,1.300587,0.888139,40.461387,2
1,ChronosZeroShot[bolt_small],-0.600741,-0.662158,1.939696,2.172404,5.104217,1


In [ ]:
predictions = predictor.predict(train_1,model='ChronosFineTuned[bolt_small]')
ufs = pd.unique(data['uf'])
predictor.plot(
    data=pd.concat([train_1,test_1]),
    quantile_levels=[0.1,0.2,0.5,0.8,0.9],
    predictions=predictions,
    item_ids = ufs,
    max_num_item_ids=None,
    max_history_length=200,
)